In [1]:
import datasets
import json
import os
import math

In [2]:
# Change these variables to perform evalution for you task
test_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'processed_data/gpt3/completion_1/processed_test.json'
generated_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_1/generated.json'

In [3]:
# Get all the target prompts and completions
test_data = []

for line in open(test_path):
    test_data.append((json.loads(line)))
    
generated_data = []
# Get the generated completions
with open(generated_path) as f:
    generated = json.load(f)

for key in generated:
    generated_data.append(generated[key])

In [4]:
# Put the predictions and gold references in lists

predictions = []
gold_references = []
predictions_list = []
gold_references_list = []

for i in range(len(test_data)):
    
    completion = test_data[i]['completion']
    clean_completion = completion.split('Question: ')[1].split('\n###')[0]
    
    generated_completion = generated_data[i]
    clean_generated_completion = generated_completion.split('Question: ')[1]
    
    predictions.append(clean_generated_completion)
    gold_references.append(clean_completion)

    predictions_list.append(clean_generated_completion.split(' '))
    gold_references_list.append([clean_completion.split(' ')])

In [5]:
bleu = datasets.load_metric('bleu')
rouge = datasets.load_metric('rouge')

In [6]:
bleu.add_batch(predictions=predictions_list, references=gold_references_list)
rouge.add_batch(predictions=predictions, references=gold_references)

In [7]:
final_bleu = bleu.compute()
final_rouge = rouge.compute()

In [8]:
# Define path of downloaded meteor from https://www.cs.cmu.edu/~alavie/METEOR/
meteor_path = "<PATH_TO_YOUR_DOWNLOADED_METEOR"

# Move results to meteor directory
with open(meteor_path + "predictions.txt", 'w') as f:
    for i in range(len(predictions)):
        f.write(predictions[i] + '\n')
        
with open(meteor_path + "ground_truth.txt", 'w') as f:
    for i in range(len(gold_references)):
        f.write(gold_references[i] + '\n')

# Run the meteor command from the meteor directory and remove result files again   
wd = os.getcwd()
os.chdir(meteor_path)
output = os.popen("java -Xmx2G -jar meteor-*.jar predictions.txt ground_truth.txt -l en -norm").read()
os.remove(meteor_path + "predictions.txt")
os.remove(meteor_path + "ground_truth.txt")
os.chdir(wd)

# Get the score from the output
meteor_score = round(float(output.split("Final score:")[1].strip()) * 100, 2)

In [9]:
# Create result files

with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_1/' + 'predictions.txt', 'w') as f:
    for i in range(len(predictions)):
        f.write(predictions[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_1/' + 'ground_truth.txt', 'w') as f:
    for i in range(len(gold_references)):
        f.write(gold_references[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_1/' + 'automatic_evaluation.txt', 'w') as f:
    f.write("BLEU: " + str(round(final_bleu['bleu'] * 100, 2)))
    f.write('\n')
    f.write("ROUGE-L: " + str(round(final_rouge['rougeL'].mid.fmeasure * 100, 2)))
    f.write('\n')
    f.write("METEOR: " + str(meteor_score))

In [10]:
count = 0
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'human_evaluation/experiment1/' + 'exp1_gpt3.txt', 'w') as f:
    for i in range(0,len(test_data),math.floor(len(test_data)/100)):
        f.write('Excel row: ' + str(count+2) + ' Test instance: ' + str(i+1) + '\n\n')
        f.write(test_data[i]['prompt'].split("\n\n###")[0] + '\n\n')
        f.write('Gold reference: ' + gold_references[i] + '\n\n')
        f.write('Generated: ' + predictions[i] + '\n')
        f.write('----------------------------------------------------------------------------------------' + '\n\n')
        count+=1
        
        if count == 100:
            break